In [13]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from data_storage import create_connection
from stockstats import StockDataFrame
from risk_metrics import Risk_Metrics

In [14]:
connection = create_connection("../database/crypto_billionairs.db")

In [15]:
def momentum(df, lag):
    return df.pct_change(periods=lag)

In [16]:
class Momentum_AR_raw_long_short(Strategy):
   
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        self.init_long_signal = self.I(init_buy_signal, self.data.df)
        self.init_close_long_signal = self.I(init_close_long_signal, self.data.df)
        self.init_short_signal = self.I(init_short_signal, self.data.df)
        self.init_close_short_signal = self.I(init_close_short_signal, self.data.df)

        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if self.init_long_signal[-1] == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()
            
        elif self.init_close_long_signal[-1] == 1 and self.position.is_long:
             self.position.close()
             
        elif self.init_short_signal[-1] == -1 and self.position.is_short:
            self.position.close()
            self.sell()
            
        elif self.init_close_short_signal[-1] == -1 and self.position.is_short:
            self.position.close()
            

In [17]:
def init_buy_signal(trash):
    return df_temp['buy_indicator'].shift(1)

def init_short_signal(trash):
    return df_temp['short_indicator'].shift(1)

def init_close_long_signal(df):
    return df_temp['close_buy_indicator'].shift(1)

def init_close_short_signal(df):
    return df_temp['close_short_indicator'].shift(1)

In [18]:
def run_backtesting_raw(db_connection):
    
    global df_temp
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    filtered_table_names = [name for name in table_names_list if "_1day_features" in name and 'trades' not in name and not 'equity_curve' in name and not "_pooling" in name and "_ensemble" not in name]
    print(filtered_table_names)
    df_risk = pd.DataFrame(columns = range(11))
    df_risk.columns = ["table_name", "timehorizon", "return", "buy and hold return", "annualized_return", "sharpe_ratio_annualized", "sortino_ratio_annualized", "maximum_drawdown", "calmar_ratio_annualized", "trades_count", "win_rate"]
    
    for table in filtered_table_names:
        
        df_temp = pd.read_sql_query(f"select * from {table}", db_connection)
        df_temp = df_temp[-365:]
        df_backtesting = pd.DataFrame()
        df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_temp[['open', 'high', 'low', 'close', 'volume']]
        
        
        bt = Backtest(df_backtesting, Momentum_AR_raw_long_short, cash=100_000, commission=.001)
        stats = bt.run()
        trades = pd.DataFrame(stats['_trades'])
        #trades.to_sql(f"trades_{table}", db_connection, if_exists="replace")
        
        #equity_curve = pd.DataFrame(stats["_equity_curve"])
        #equity_curve.to_sql(f"{table}_equity_curve", db_connection, if_exists="replace")
        
        risk = Risk_Metrics(trades, df_temp, 0, stats)
        risk_metrics_list = [f"{table}", len(df_backtesting), stats["Return [%]"], stats["Buy & Hold Return [%]"], stats["Return [%]"],
                             risk.sharpe_ratio(), risk.sortino_ratio(),
                             risk.max_drawdown(), risk.calmar_ratio(), len(trades), stats["Win Rate [%]"]]
        print(risk_metrics_list)
        #df_risk.append(risk_metrics_list)
        df_risk.loc[len(df_risk)] = risk_metrics_list
        
    
    df_risk.to_sql("cryptocurrencies_risk_metrics_1m_abnormal_day", db_connection, if_exists="replace")

In [19]:
run_backtesting_raw(connection)

['ADA_1min_complete_1day_preprocessed_1day_features', 'BCH_1min_complete_1day_preprocessed_1day_features', 'BNT_1min_complete_1day_preprocessed_1day_features', 'BSV_1min_complete_1day_preprocessed_1day_features', 'BTC_1min_complete_1day_preprocessed_1day_features', 'BTG_1min_complete_1day_preprocessed_1day_features', 'DASH_1min_complete_1day_preprocessed_1day_features', 'DOGE_1min_complete_1day_preprocessed_1day_features', 'EOS_1min_complete_1day_preprocessed_1day_features', 'ETC_1min_complete_1day_preprocessed_1day_features', 'ETH_1min_complete_1day_preprocessed_1day_features', 'FUN_1min_complete_1day_preprocessed_1day_features', 'ICX_1min_complete_1day_preprocessed_1day_features', 'KNC_1min_complete_1day_preprocessed_1day_features', 'LINK_1min_complete_1day_preprocessed_1day_features', 'LRC_1min_complete_1day_preprocessed_1day_features', 'LTC_1min_complete_1day_preprocessed_1day_features', 'MKR_1min_complete_1day_preprocessed_1day_features', 'NEO_1min_complete_1day_preprocessed_1day_

<ipython-input-18-f781cfb66b31>:22: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_backtesting, Momentum_AR_raw_long_short, cash=100_000, commission=.001)



['BNT_1min_complete_1day_preprocessed_1day_features', 365, 64.77305002450044, -64.90461253663804, 64.77305002450044, 9.662544419857705, 1.389047032847732, 0.26206031598489116, 2.4716848020680424, 38, 63.1578947368421]
76.86894167770055
['BSV_1min_complete_1day_preprocessed_1day_features', 365, 76.86894167770055, -61.065740289289785, 76.86894167770055, 9.702261593565698, 1.5692386983798152, 0.16872348345138719, 4.555912437633385, 40, 52.5]
12.802699726480379
['BTC_1min_complete_1day_preprocessed_1day_features', 365, 12.802699726480379, -21.536649743714253, 12.802699726480379, 2.9322257461349115, 0.26036409451327847, 0.26460079065690284, 0.48384963985542745, 44, 45.45454545454545]
7.982171997100347
['BTG_1min_complete_1day_preprocessed_1day_features', 365, 7.982171997100347, 1.760497957266667, 7.982171997100347, 1.1120628843794738, 0.16590659554322115, 0.2683687102727155, 0.29743303490890893, 37, 51.35135135135135]
124.16226659860061
['DASH_1min_complete_1day_preprocessed_1day_features'

In [20]:
class Momentum_AR_long_short(Strategy):
   
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        self.init_long_short_signal = self.I(init_long_short_signal, self.data.df)
        self.init_close_long_short_signal = self.I(init_close_long_short_signal, self.data.df)
       

        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if int(float(self.init_long_short_signal[-1])) == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()
            
        elif int(float(self.init_close_long_short_signal[-1])) == 1 and self.position.is_long:
             self.position.close()
            
        elif int(float(self.init_long_short_signal[-1])) == -1 and self.position.is_short is False:
            self.position.close()
            self.sell()
            
        elif int(float(self.init_close_long_short_signal[-1])) == -1 and self.position.is_short:
            self.position.close()

In [21]:
def init_long_short_signal(some_input):
    return df_temp_ml['buy_short_indicator']

def init_close_long_short_signal(some_input):
    return df_temp_ml['close_buy_short_indicator']

In [49]:
def run_backtesting_ml(db_connection):
    
    global df_temp_ml
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    #filtered_table_names = [name for name in table_names_list if ("_ensemble" in name or "_svc" in name or "_knn" in name or "_random_forest" in name or "_logistic_regression" in name or "_mlp" in name) and 'trades' not in name and not 'equity_curve' in name]
    filtered_table_names = [name for name in table_names_list if "no_" in name]
    df_risk = pd.DataFrame(columns = range(11))
    df_risk.columns = ["table_name", "timehorizon", "return", "buy and hold return", "annualized_return", "sharpe_ratio_annualized", "sortino_ratio_annualized", "maximum_drawdown", "calmar_ratio_annualized", "trades_count", "win_rate"]
    
    for table in filtered_table_names:
        
        df_temp_ml = pd.read_sql_query(f"select * from {table}", db_connection)
        df_temp_ml["return"] = momentum(df_temp_ml["close"], 1) + 1
        
        #df_temp_ml = df_temp_ml[-356:]
        
        df_backtesting = pd.DataFrame()
        df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_temp_ml[['open', 'high', 'low', 'close', 'volume']]
        
        
        bt = Backtest(df_backtesting, Momentum_AR_long_short, cash=100_000, commission=.001)
        stats = bt.run()
        trades = pd.DataFrame(stats['_trades'])
        #trades.to_sql(f"trades_{table}", db_connection, if_exists="replace")
        
        #equity_curve = pd.DataFrame(stats["_equity_curve"])
        #equity_curve.to_sql(f"{table}_equity_curve", db_connection, if_exists="replace")
        
        risk = Risk_Metrics(trades, df_temp_ml, 0, stats)
        
        try:
            risk_metrics_list = [f"{table}", len(df_backtesting), stats["Return [%]"], stats["Buy & Hold Return [%]"], risk._annualize(stats["Return [%]"]),
                             risk.sharpe_ratio(), risk.sortino_ratio(),
                             risk.max_drawdown(), risk.calmar_ratio(), len(trades), stats["Win Rate [%]"]]
            print(risk_metrics_list)
            df_risk.loc[len(df_risk)] = risk_metrics_list
        
        except:
            print(table, len(trades))
            pass
        
    
    df_risk.to_sql("cryptocurrencies_risk_metrics_1m_abnormal_day_ml", db_connection, if_exists="replace")

In [51]:
run_backtesting_ml(connection)

<ipython-input-49-bf9c2ac30c19>:25: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_backtesting, Momentum_AR_long_short, cash=100_000, commission=.001)


981303.7700453498
['no_0_threshold_ensemble_long_ADA_1min_complete_1day_preprocessed_1day_features', 733, 981303.7700453498, 2398.225358135557, 963.0308476827371, 905.9744890828563, 220.9259910205347, 0.048024489659100396, 204334.03395040508, 235, 69.7872340425532]
268372.5943950826
['no_0_threshold_ensemble_long_BCH_1min_complete_1day_preprocessed_1day_features', 1207, 268372.5943950826, -67.83141905148277, 43.81923725089142, 2.5344577437607345, 23.03453930472919, 0.12038024105637098, 22293.74123527553, 322, 58.38509316770186]
17594.394908771905
['no_0_threshold_ensemble_long_BNT_1min_complete_1day_preprocessed_1day_features', 108, 17594.394908771905, 909.6933100270139, 222737270460810.2, 78999502.13709557, 82858113.21076426, 0.10161513761046641, 1731.4738062175957, 30, 63.33333333333333]
16087.812972431422
['no_0_threshold_ensemble_long_BSV_1min_complete_1day_preprocessed_1day_features', 730, 16087.812972431422, 487.6582705073533, 126.83774269684643, 50.71152386298224, 27.21505322462